In [1]:
!pip install -r requirements.txt

In [1]:
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain_community.chains.graph_qa.cypher import GraphCypherQAChain

from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

load_dotenv()

True

In [2]:
# Neo4j Bağlantı Bilgilerim
uri = os.getenv("NEO4J_URI")
user = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")

In [3]:
uri

'bolt://localhost:7687'

In [4]:
# Neo4j Graph objesi oluşturma
graph = Neo4jGraph(
    url=uri,
    username=user,
    password=password
)

C:\Users\HP\AppData\Local\Temp\ipykernel_11248\4249525564.py:2: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


ValueError: Could not use APOC procedures. Please ensure the APOC plugin is installed in Neo4j and that 'apoc.meta.data()' is allowed in Neo4j configuration 

In [5]:
# OpenAI API Anahtarı
openai_api_key = os.getenv("OPENAI_API_KEY")

In [10]:
import langchain
langchain.verbose = False
langchain.debug = False
langchain.llm_cache = False

# hata verdiği icin

In [6]:
# LLM ve Embedding Modeli

llm = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4.1-nano-2025-04-14"
)
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [54]:
result = graph.query("MATCH (n) RETURN DISTINCT labels(n) AS labels, count(*) AS adet ORDER BY adet DESC")
for row in result:
    print(row)

{'labels': ['Person'], 'adet': 16}
{'labels': ['Article'], 'adet': 3}
{'labels': ['State'], 'adet': 3}
{'labels': ['Location'], 'adet': 2}
{'labels': ['Political party'], 'adet': 1}
{'labels': ['Building'], 'adet': 1}
{'labels': ['Organization'], 'adet': 1}


In [53]:
# # Graph şemasını al (Otomatik)
try:
    schema = graph.get_schema # İşte sihirli satır!
    print("--- Otomatik Çekilen Graph Şeması ---")
    print(schema)
    print("-------------------------------------\n")
except Exception as e:
    print(f"Otomatik şema çekilirken hata oluştu: {e}\n")
    schema = "Otomatik şema alınamadı."

--- Otomatik Çekilen Graph Şeması ---
Node properties:
Article {neo4jImportId: STRING, date: STRING, id: STRING, text: STRING}
Person {neo4jImportId: STRING, id: STRING, born: INTEGER, name: STRING}
Location {neo4jImportId: STRING, id: STRING}
Political party {neo4jImportId: STRING, id: STRING}
Building {neo4jImportId: STRING, id: STRING}
Organization {neo4jImportId: STRING, id: STRING}
State {neo4jImportId: STRING, id: STRING}
Relationship properties:

The relationships:
(:Article)-[:HAS_ENTITY]->(:Person)
(:Article)-[:HAS_ENTITY]->(:Political party)
(:Article)-[:HAS_ENTITY]->(:Building)
(:Article)-[:HAS_ENTITY]->(:Location)
(:Article)-[:HAS_ENTITY]->(:Organization)
(:Person)-[:PARTICIPATED_IN_COMPETITION]->(:Location)
(:Person)-[:SKIPPED_PRIMARIES]->(:Person)
(:Person)-[:COMMENTED_ON]->(:Person)
(:Person)-[:COMMENTED_ON]->(:Location)
(:Person)-[:LOST_TO]->(:Person)
(:Person)-[:RANKED_THIRD_TO]->(:Person)
(:Person)-[:WON_PRIMARY]->(:State)
(:Person)-[:NEXT_FOCUS]->(:State)
(:Person)-[

Otomatik olarak schema'yı almak

Avantajları:

Hızlı ve Kolay: Tek satır kodla temel şemayı alırsınız.

Güncel: Veritabanınız değiştikçe otomatik olarak güncel şemayı çeker.

Dezavantajları:

Yetersiz Detay: Bazen LLM'in daha iyi anlaması için yeterli detayı sunmayabilir. Örneğin, bir özelliğin "String" mi "Float" mı olduğu, hangi değerleri alabileceği veya bir id alanının UNIQUE olup olmadığı gibi bilgiler bu otomatik şemada her zaman açıkça belirtilmeyebilir.

Otomatik şema iyi bir başlangıç olsa da, LLM'e daha fazla "ipucu" ve "açıklama" vermek, onun sorguları daha doğru anlamasına ve daha iyi Cypher sorguları üretmesine yardımcı olur. Bu, özellikle projenizdeki varlıkların ve ilişkilerin özel anlamları varsa çok önemlidir.


In [7]:
# 2. Detaylı Manuel Şema Tanımlaması
# LLM'in sorguları daha iyi anlaması ve daha doğru Cypher üretmesi için
# şemayı doğal dilde, daha açıklayıcı bir şekilde tanımlayabilirsiniz.
# Bu, özellikle node'ların ve ilişkilerin anlamsal rollerini vurgulamak
# veya belirli özelliklerin ne anlama geldiğini açıklamak için önemlidir.

# Sizin sağladığınız Cypher komutlarına ve CSV yapılarına göre detaylı bir şema:
detailed_schema_description = """
Graph veritabanı yemek tarifleri, malzemeler, tatlar ve pişirme teknikleri hakkında bilgi içerir.

**Node Etiketleri (Düğüm Türleri):**

1.  **Recipe (Tarif):**
    *   `id`: Tarif için benzersiz bir tanımlayıcı (String, Zorunlu, UNIQUE).
    *   `name`: Tarifin adı (String). Örnek: "Sütlaç", "Kek".
    *   `difficulty`: Tarifin zorluk derecesi (String). Olası değerler: "Kolay", "Orta", "Zor".
    *   `time_minutes`: Tarifin hazırlanma süresi (dakika cinsinden Float). Örnek: 45.0, 60.0.
    *   *Ek Etiketler (Koşullu):*
        *   `SweetDish`: Tarif "Tatlı" lezzetine sahipse bu etiket eklenir.
        *   `CreamyDish`: Tarif "Kremamsı" lezzetine sahipse bu etiket eklenir.

2.  **Ingredient (Malzeme):**
    *   `id`: Malzeme için benzersiz bir tanımlayıcı (String, Zorunlu, UNIQUE).
    *   `name`: Malzemenin adı (String). Örnek: "Şeker", "Süt", "Un".
    *   `category`: Malzemenin kategorisi (String). Örnek: "Süt Ürünü", "Tahıl", "Tatlandırıcı".
    *   *Ek Etiketler (Koşullu):*
        *   `Substitute`: Bu malzeme başka bir malzemenin yerine kullanılabiliyorsa bu etiket eklenir.

3.  **Flavor (Lezzet/Tat):**
    *   `id`: Lezzet için benzersiz bir tanımlayıcı (String, Zorunlu, UNIQUE).
    *   `name`: Lezzetin adı (String). Örnek: "Tatlı", "Kremamsı", "Karamelli".

4.  **Technique (Pişirme Tekniği):**
    *   `id`: Teknik için benzersiz bir tanımlayıcı (String, Zorunlu, UNIQUE).
    *   `name`: Pişirme tekniğinin adı (String). Örnek: "Fırınlama", "Kaynatma", "Karıştırma", "Soğutma".

**İlişki Türleri:**

1.  **CONTAINS (İçerir):** Bir tarifin hangi malzemeleri içerdiğini gösterir.
    *   Başlangıç Düğümü: `Recipe`
    *   Bitiş Düğümü: `Ingredient`
    *   Yön: `(Recipe)-[:CONTAINS]->(Ingredient)`
    *   Örnek: (Sütlaç)-[:CONTAINS]->(Şeker)

2.  **HAS_FLAVOR (Lezzete Sahip):** Bir tarifin hangi lezzet profiline sahip olduğunu gösterir.
    *   Başlangıç Düğümü: `Recipe`
    *   Bitiş Düğümü: `Flavor`
    *   Yön: `(Recipe)-[:HAS_FLAVOR]->(Flavor)`
    *   Örnek: (Sütlaç)-[:HAS_FLAVOR]->(Tatlı)

3.  **USES_TECHNIQUE (Teknik Kullanır):** Bir tarifin hazırlanmasında hangi pişirme tekniklerinin kullanıldığını gösterir.
    *   Başlangıç Düğümü: `Recipe`
    *   Bitiş Düğümü: `Technique`
    *   Yön: `(Recipe)-[:USES_TECHNIQUE]->(Technique)`
    *   Örnek: (Kek)-[:USES_TECHNIQUE]->(Fırınlama)

4.  **CAN_REPLACE (Yerine Kullanılabilir):** Bir malzemenin başka bir malzemenin yerine alternatif olarak kullanılabileceğini gösterir.
    *   Başlangıç Düğümü: `Ingredient`
    *   Bitiş Düğümü: `Ingredient`
    *   Yön: `(Ingredient)-[:CAN_REPLACE]->(Ingredient)`
    *   Örnek: (Bal)-[:CAN_REPLACE]->(Şeker)

**Önemli Notlar:**
*   Tüm `id` alanları, ilgili düğüm tipi için benzersizliği garanti eden kısıtlamalara (constraints) sahiptir.
*   `Recipe` düğümlerindeki `time_minutes` özelliği sayısal (float) bir değerdir.
"""

print("--- Detaylı Manuel Tanımlanmış Graph Şeması ---")
print(detailed_schema_description)
print("-----------------------------------------------\n")

# Bu detaylı şemayı LLM zincirlerinize (örneğin GraphCypherQAChain) iletebilirsiniz.
# Genellikle zincir oluşturulurken `graph_schema` veya benzer bir parametre ile bu bilgi verilir
# ya da prompt'un bir parçası olarak LLM'e sunulur.

# Örnek: GraphCypherQAChain'e detaylı şemayı verme (kullanım senaryosuna göre uyarlanabilir)
# from langchain.chains import GraphCypherQAChain
#
# cypher_qa_chain_detailed_schema = GraphCypherQAChain.from_llm(
#     llm=llm,
#     graph=graph, # Neo4jGraph objesi
#     verbose=True,
#     graph_schema=detailed_schema_description # Detaylı şemayı burada belirtiyoruz
#     # Bazı Langchain versiyonlarında/yapılarında bu doğrudan parametre olmayabilir,
#     # o zaman prompt template'ine eklenmesi gerekebilir.
# )
#
# soru_detayli_sema_ile = "Tatlı lezzetine sahip ve fırınlama tekniği kullanılan tarifler hangileridir?"
# yanıt_detayli_sema_ile = cypher_qa_chain_detailed_schema.invoke({"query": soru_detayli_sema_ile})
# print(f"Soru (Detaylı Şema ile): {soru_detayli_sema_ile}\nYanıt: {yanıt_detayli_sema_ile['result']}\n")


--- Detaylı Manuel Tanımlanmış Graph Şeması ---

Graph veritabanı yemek tarifleri, malzemeler, tatlar ve pişirme teknikleri hakkında bilgi içerir.

**Node Etiketleri (Düğüm Türleri):**

1.  **Recipe (Tarif):**
    *   `id`: Tarif için benzersiz bir tanımlayıcı (String, Zorunlu, UNIQUE).
    *   `name`: Tarifin adı (String). Örnek: "Sütlaç", "Kek".
    *   `difficulty`: Tarifin zorluk derecesi (String). Olası değerler: "Kolay", "Orta", "Zor".
    *   `time_minutes`: Tarifin hazırlanma süresi (dakika cinsinden Float). Örnek: 45.0, 60.0.
    *   *Ek Etiketler (Koşullu):*
        *   `SweetDish`: Tarif "Tatlı" lezzetine sahipse bu etiket eklenir.
        *   `CreamyDish`: Tarif "Kremamsı" lezzetine sahipse bu etiket eklenir.

2.  **Ingredient (Malzeme):**
    *   `id`: Malzeme için benzersiz bir tanımlayıcı (String, Zorunlu, UNIQUE).
    *   `name`: Malzemenin adı (String). Örnek: "Şeker", "Süt", "Un".
    *   `category`: Malzemenin kategorisi (String). Örnek: "Süt Ürünü", "Tahıl", "Tatlan

In [39]:
# Soru-Cevap Zinciri Oluşturma
cypher_qa_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True,
    allow_dangerous_requests= True
)

# Doğal dilde sorgular yapma
soru1 = "Sütlaç hangi malzemeleri içerir?"
yanit1 = cypher_qa_chain.invoke({"query": soru1})
print(f"Soru: {soru1}\nYanıt: {yanit1['result']}\n")

soru2 = "Fırınlama tekniği kullanan ve zorluk derecesi 'Zor' olan tarifler hangileridir?"
yanit2 = cypher_qa_chain.invoke({"query": soru2})
print(f"Soru: {soru2}\nYanıt: {yanit2['result']}\n")

soru3 = "Kremamsı tada sahip tarifler nelerdir?"
yanit3 = cypher_qa_chain.invoke({"query": soru3})
print(f"Soru: {soru3}\nYanıt: {yanit3['result']}\n")

soru4 = "Şeker yerine kullanılabilecek malzemeler nelerdir?"
yanit4 = cypher_qa_chain.invoke({"query": soru4})
print(f"Soru: {soru4}\nYanıt: {yanit4['result']}\n")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Article)-[:HAS_ENTITY]->(e)
WHERE e:Person OR e:Political party OR e:Building OR e:Location OR e:Organization
RETURN DISTINCT e



CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'party': expected an expression, 'FOREACH', 'ORDER BY', 'CALL', 'CREATE', 'LOAD CSV', 'DELETE', 'DETACH', 'FINISH', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REMOVE', 'RETURN', 'SET', 'SKIP', 'UNION', 'UNWIND', 'USE', 'WITH' or <EOF> (line 3, column 31 (offset: 74))
"WHERE e:Person OR e:Political party OR e:Building OR e:Location OR e:Organization"
                               ^}

## 5. Kendi Zincirlerinizi (Chains) Oluşturma
Daha spesifik görevler için kendi özel zincirlerinizi oluşturabilirsiniz. Örneğin, eldeki malzemelere göre tarif öneren bir zincir:

In [40]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Malzemelere göre tarif öneren zincir
prompt_template = PromptTemplate(
    input_variables=["malzemeler", "schema"],
    template="""
    Sen bir yemek tarifi uzmanısın ve bir graph veritabanına erişimin var.
    Graph Şeması:
    {schema}

    Kullanıcının elinde şu malzemeler var: {malzemeler}.
    Bu malzemelerin çoğunu veya tamamını içeren, veritabanındaki tariflerden 2 tane öner.
    Sadece tarif isimlerini ve neden önerdiğini kısaca belirt.
    Eğer uygun tarif bulamazsan, bulamadığını belirt.
    Cypher sorgusunu şu formatta üret:
    ```
    MATCH (r:Recipe)-[:CONTAINS]->(i:Ingredient)
    WHERE i.name IN $malzeme_listesi
    // Diğer koşullar ve return ifadesi
    ```
    """
)

def malzemeler_ile_tarif_bul_ve_oner(user_input: dict):
    malzemeler_listesi = [m.strip() for m in user_input["malzemeler"].split(',')]

    # LLM'den Cypher sorgusu üretmesini isteyelim (basit bir örnek, daha karmaşık hale getirilebilir)
    # Gerçek bir uygulamada, bu kısım daha sofistike bir LLM çağrısı veya GraphCypherQAChain benzeri bir yapı olabilir.
    # Şimdilik, örnek bir sorgu kullanalım.
    # Gerçekçi bir senaryoda, LLM'e malzemeleri verip uygun bir sorgu pattern'i oluşturmasını isteyebiliriz.
    # Bu örnekte, malzemelerin çoğunu içeren tarifleri bulmaya yönelik bir Cypher sorgusu manuel oluşturulmuştur.
    # LLM'in bu sorguyu üretmesi hedeflenmelidir.

    # Örnek Cypher sorgusu (LLM tarafından üretilmesi hedeflenir)
    # Bu sorgu, verilen malzemelerden en az X tanesini içeren tarifleri bulur.
    # Daha gelişmiş bir LLM prompt'u ile bu sorgu dinamik olarak üretilebilir.
    cypher_query = f"""
    MATCH (r:Recipe)-[:CONTAINS]->(i:Ingredient)
    WHERE i.name IN {malzemeler_listesi}
    WITH r, collect(i.name) AS recipe_ingredients, {malzemeler_listesi} AS available_ingredients
    WITH r, recipe_ingredients, available_ingredients,
         [x IN recipe_ingredients WHERE x IN available_ingredients] AS common_ingredients
    WHERE size(common_ingredients) > 0 // En az bir ortak malzeme
    RETURN r.name AS recipe_name, size(common_ingredients) AS common_ingredient_count, recipe_ingredients
    ORDER BY common_ingredient_count DESC
    LIMIT 5
    """
    try:
        results = graph.query(cypher_query)
        if not results:
            return "Belirttiğiniz malzemelerle eşleşen tarif bulunamadı."

        öneriler = "Bulunan tarifler:\n"
        for record in results:
            öneriler += f"- {record['recipe_name']} ({record['common_ingredient_count']} ortak malzeme: {record['recipe_ingredients']})\n"
        return öneriler
    except Exception as e:
        return f"Sorgu çalıştırılırken hata oluştu: {e}"


malzeme_tarif_zinciri = (
    {
        "malzemeler": RunnablePassthrough(), 
        "schema": lambda x: graph.get_schema 
    }
    | RunnablePassthrough.assign(öneri=malzemeler_ile_tarif_bul_ve_oner)
)



# Zinciri çalıştırma
kullanici_malzemeleri = "Süt, Şeker, Pirinç"
yanit = malzeme_tarif_zinciri.invoke(kullanici_malzemeleri)
print(f"Elimdeki malzemeler: {kullanici_malzemeleri}\nÖneriler:\n{yanit['öneri']}")

# kullanici_malzemeleri_2 = "Un, Yumurta, Tereyağı"
# yanit_2 = malzeme_tarif_zinciri.invoke(kullanici_malzemeleri_2)
# print(f"Elimdeki malzemeler: {kullanici_malzemeleri_2}\nÖneriler:\n{yanit_2['öneri']}")


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: CONTAINS)} {position: line: 2, column: 24, offset: 24} for query: "\n    MATCH (r:Recipe)-[:CONTAINS]->(i:Ingredient)\n    WHERE i.name IN ['Süt', 'Şeker', 'Pirinç']\n    WITH r, collect(i.name) AS recipe_ingredients, ['Süt', 'Şeker', 'Pirinç'] AS available_ingredients\n    WITH r, recipe_ingredients, available_ingredients,\n         [x IN recipe_ingredients WHERE x IN available_ingredients] AS common_ingredients\n    WHERE size(common_ingredients) > 0 // En az bir ortak malzeme\n    RETURN r.name AS recipe_name, size(common_ingredien

Elimdeki malzemeler: Süt, Şeker, Pirinç
Öneriler:
Belirttiğiniz malzemelerle eşleşen tarif bulunamadı.
